# Import data for Wunderink_2021/Grant:

In [1]:
import scanpy as sc
import numpy as np
import pandas as pd

# Load data

In [2]:
!pwd

/mnt/znas/icb_zstore01/groups/ml01/workspace/hlca_lisa.sikkema_malte.luecken/query_datasets


In [3]:
adata = sc.read('raw/Grant/GSE155249_main.h5ad')

In [4]:
gene_set = pd.read_csv('genes_for_mapping.csv')

In [5]:
meta = pd.read_csv('raw/Grant/patients-metadata.csv', index_col=0)

In [6]:
raw_data = dict()
files = !ls raw/Grant/*.h5
for file in files:
    samp = file.split('_')[2]
    raw_data[samp] = sc.read_10x_h5(file)
    raw_data[samp].var_names_make_unique(join='.')

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Vari

In [7]:
adata

AnnData object with n_obs × n_vars = 77146 × 21819
    obs: 'Patient', 'Day after intubation', 'Sample type', 'Sample', 'Sample status', 'Cluster', 'COVID-19', 'No of SARS-CoV-2 genes', 'IFNA Response', 'IFNG Response', 'No of genes', 'No of UMIs', '% of mito genes', '% of ribo genes'
    var: 'gene_ids-0', 'feature_types-0', 'n_cells-0', 'gene_ids-1', 'feature_types-1', 'n_cells-1', 'gene_ids-10', 'feature_types-10', 'n_cells-10', 'gene_ids-11', 'feature_types-11', 'n_cells-11', 'gene_ids-12', 'feature_types-12', 'n_cells-12', 'gene_ids-13', 'feature_types-13', 'n_cells-13', 'gene_ids-14', 'feature_types-14', 'n_cells-14', 'gene_ids-15', 'feature_types-15', 'n_cells-15', 'gene_ids-16', 'feature_types-16', 'n_cells-16', 'gene_ids-17', 'feature_types-17', 'n_cells-17', 'gene_ids-18', 'feature_types-18', 'n_cells-18', 'gene_ids-2', 'feature_types-2', 'n_cells-2', 'gene_ids-3', 'feature_types-3', 'n_cells-3', 'gene_ids-4', 'feature_types-4', 'n_cells-4', 'gene_ids-5', 'feature_types-5', '

# Make object lightweight

In [8]:
del adata.uns
del adata.obsm
del adata.varm
del adata.obsp

# Clean adata.var


In [9]:
to_drop = [col for col in adata.var.columns if 'feature_types' in col]
for i in to_drop:
    del adata.var[i]

In [10]:
gene_id_cols = [col for col in adata.var.columns if 'gene_ids' in col]

def get_id(series):
    series_sub = series[~series.isin(['nan'])]
    return series_sub.value_counts().index[0]

adata.var['gene_id'] = adata.var[gene_id_cols].apply(get_id, axis=1)

for i in gene_id_cols:
    del adata.var[i]

In [11]:
n_cells_col = [col for col in adata.var.columns if 'n_cells' in col]

for i in n_cells_col:
    del adata.var[i]

In [12]:
to_drop = ['mito', 'ribo', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection', 'mean', 'std']
adata.var = adata.var.drop(columns=to_drop)

In [13]:
adata

AnnData object with n_obs × n_vars = 77146 × 21819
    obs: 'Patient', 'Day after intubation', 'Sample type', 'Sample', 'Sample status', 'Cluster', 'COVID-19', 'No of SARS-CoV-2 genes', 'IFNA Response', 'IFNG Response', 'No of genes', 'No of UMIs', '% of mito genes', '% of ribo genes'
    var: 'gene_id'

# Organize obs columns

In [14]:
adata.obs['subject_ID'] = adata.obs['Patient']
del adata.obs['Patient']

In [15]:
adata.obs['sample'] = adata.obs['Sample']
del adata.obs['Sample']

In [16]:
adata.obs['sample_type'] = adata.obs['Sample type']
del adata.obs['Sample type']

In [17]:
adata.obs['original_celltype_ann'] = adata.obs['Cluster']
del adata.obs['Cluster']

In [18]:
adata.obs['condition'] = ['COVID-19' for i in range(adata.n_obs)]
del adata.obs['COVID-19']

In [19]:
adata.obs['n_counts'] = adata.obs['No of UMIs']
del adata.obs['No of UMIs']

In [20]:
adata.obs['n_genes'] = adata.obs['No of genes']
del adata.obs['No of genes']

In [21]:
adata.obs['study'] = ['Wunderink2021' for i in range(adata.n_obs)]

In [22]:
adata

AnnData object with n_obs × n_vars = 77146 × 21819
    obs: 'Day after intubation', 'Sample status', 'No of SARS-CoV-2 genes', 'IFNA Response', 'IFNG Response', '% of mito genes', '% of ribo genes', 'subject_ID', 'sample', 'sample_type', 'original_celltype_ann', 'condition', 'n_counts', 'n_genes', 'study'
    var: 'gene_id'

# Match raw count data to processed object

In [23]:
# Assign a sample ID
for samp in raw_data.keys():
    raw_data[samp].obs['sample'] = [samp for i in range(raw_data[samp].n_obs)]

In [24]:
# Match obs_names to merged adata object
for samp in raw_data.keys():
    raw_data[samp].obs_names = ['-'.join([idx.split('-')[0],str(int(samp)-1)])  for idx in raw_data[samp].obs_names]

In [25]:
# Merge the raw objects
samples = list(raw_data.keys())
raw_merged = raw_data[samples.pop(0)]

for i in samples:
    raw_merged = raw_merged.concatenate(raw_data[i], index_unique=None)

In [26]:
raw_merged.var_names = [i.split('GRCh38______')[-1] for i in raw_merged.var_names]

In [27]:
set(adata.var_names).difference(set(raw_merged.var_names))

{'SARS-CoV-2-Antisense',
 'SARS-CoV-2-E',
 'SARS-CoV-2-M',
 'SARS-CoV-2-N',
 'SARS-CoV-2-ORF10',
 'SARS-CoV-2-ORF1ab',
 'SARS-CoV-2-ORF3a',
 'SARS-CoV-2-ORF6',
 'SARS-CoV-2-ORF7a',
 'SARS-CoV-2-ORF7b',
 'SARS-CoV-2-ORF8',
 'SARS-CoV-2-S'}

In [28]:
# Fix SARS-CoV-2 gene names
raw_merged.var_names = [i.replace('_', '-') for i in raw_merged.var_names]
raw_merged.var_names = [i.replace('2i', '2') for i in raw_merged.var_names]
raw_merged.var_names = [i.replace('SARS-CoV-2-SARS-CoV-2-antisense', 'SARS-CoV-2-Antisense') for i in raw_merged.var_names]

In [29]:
set(adata.var_names).difference(set(raw_merged.var_names))
len(set(adata.var_names).intersection(set(raw_merged.var_names)))

set()

21819

In [30]:
adata

AnnData object with n_obs × n_vars = 77146 × 21819
    obs: 'Day after intubation', 'Sample status', 'No of SARS-CoV-2 genes', 'IFNA Response', 'IFNG Response', '% of mito genes', '% of ribo genes', 'subject_ID', 'sample', 'sample_type', 'original_celltype_ann', 'condition', 'n_counts', 'n_genes', 'study'
    var: 'gene_id'

# Assign raw count data to merged object

In [31]:
raw_merged_sub = raw_merged[adata.obs_names,:][:,adata.var_names].copy()
raw_merged_sub

/home/icb/malte.luecken/anaconda3/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2492: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


AnnData object with n_obs × n_vars = 77146 × 21819
    obs: 'sample', 'batch'
    var: 'gene_ids', 'feature_types', 'genome'

In [32]:
adata.layers['counts'] = raw_merged_sub.X

### Check raw count mapping

In [33]:
adata.layers['counts'][20:30,10:20].A
adata.X[20:30,10:20]
raw_merged_sub.X[20:30,10:20].A

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32)

array([[-0.00844672, -0.02548061, -0.43014944, -0.30342752, -0.44830588,
        -0.27202973, -0.4466437 , -0.01194746, -0.2808309 , -0.00892327],
       [-0.00844672, -0.02548061, -0.43014944, -0.30342752, -0.44830588,
        -0.27202973, -0.4466437 , -0.01194746, -0.2808309 , -0.00892327],
       [-0.00844672, -0.02548061, -0.43014944, -0.30342752, -0.44830588,
        -0.27202973,  3.1880455 , -0.01194746, -0.2808309 , -0.00892327],
       [-0.00844672, -0.02548061,  3.579508  , -0.30342752, -0.44830588,
        -0.27202973, -0.4466437 , -0.01194746, -0.2808309 , -0.00892327],
       [-0.00844672, -0.02548061, -0.43014944, -0.30342752, -0.44830588,
        -0.27202973, -0.4466437 , -0.01194746, -0.2808309 , -0.00892327],
       [-0.00844672, -0.02548061, -0.43014944, -0.30342752, -0.44830588,
        -0.27202973, -0.4466437 , -0.01194746, -0.2808309 , -0.00892327],
       [-0.00844672, -0.02548061, -0.43014944, -0.30342752,  2.8370743 ,
        -0.27202973, -0.4466437 , -0.01194746

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32)

In [34]:
adata.X = adata.layers['counts']
del adata.layers['counts']

# Assign additional patient metadata

In [35]:
adata

AnnData object with n_obs × n_vars = 77146 × 21819
    obs: 'Day after intubation', 'Sample status', 'No of SARS-CoV-2 genes', 'IFNA Response', 'IFNG Response', '% of mito genes', '% of ribo genes', 'subject_ID', 'sample', 'sample_type', 'original_celltype_ann', 'condition', 'n_counts', 'n_genes', 'study'
    var: 'gene_id'

In [36]:
meta

,age,sex,diagnosis,days_from_first_intubation,scRNAseq_id,superinfection
1,38.310062,Male,COVID-19,2,1,Primary Only
2,77.176705,Male,COVID-19,0,2,Primary Only
3,72.527721,Male,COVID-19,1,3,Primary Only
4,64.097878,Male,COVID-19,1,4,Primary Only
5,37.737965,Male,COVID-19,1,5,Primary Only
6,39.586014,Male,Other Pneumonia,1,6,NaN
7,70.370294,Female,COVID-19,0,7,Superinfection
8,21.116359,Male,COVID-19,0,8,Primary Only
9,61.655829,Female,COVID-19,1,9,Primary Only
10,62.370294,Male,COVID-19,0,A,Superinfection


In [37]:
meta.index = meta.scRNAseq_id

In [38]:
meta

,age,sex,diagnosis,days_from_first_intubation,scRNAseq_id,superinfection
scRNAseq_id,,,,,,
1,38.310062,Male,COVID-19,2,1,Primary Only
2,77.176705,Male,COVID-19,0,2,Primary Only
3,72.527721,Male,COVID-19,1,3,Primary Only
4,64.097878,Male,COVID-19,1,4,Primary Only
5,37.737965,Male,COVID-19,1,5,Primary Only
6,39.586014,Male,Other Pneumonia,1,6,NaN
7,70.370294,Female,COVID-19,0,7,Superinfection
8,21.116359,Male,COVID-19,0,8,Primary Only
9,61.655829,Female,COVID-19,1,9,Primary Only


In [39]:
adata.obs['subject_ID'].value_counts()

8    16099
A    15373
9    10374
3     9353
1     7060
B     4956
7     4887
5     3909
4     2756
2     2379
Name: subject_ID, dtype: int64

In [40]:
# Map patient metadata to adata order
age_list = [meta.loc[idx]['age'] for idx in adata.obs['subject_ID']] 
sex_list = [meta.loc[idx]['sex']  for idx in adata.obs['subject_ID']]
sup_list = [meta.loc[idx]['superinfection'] for idx in adata.obs['subject_ID']]
intub_list = [meta.loc[idx]['days_from_first_intubation'] for idx in adata.obs['subject_ID']]

In [41]:
adata.obs['age'] = age_list
adata.obs['sex'] = sex_list
adata.obs['superinfection'] = sup_list
adata.obs['days_from_first_intubation'] = intub_list

In [42]:
pd.crosstab(adata.obs.subject_ID, adata.obs['age'])
pd.crosstab(adata.obs.subject_ID, adata.obs['sex'])
pd.crosstab(adata.obs.subject_ID, adata.obs['superinfection'])
pd.crosstab(adata.obs.subject_ID, adata.obs['days_from_first_intubation'])

age,21.116359,37.737965,38.310062,48.054187,61.655829,62.370294,64.097878,70.370294,72.527721,77.176705
subject_ID,,,,,,,,,,
1,0,0,7060,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,2379
3,0,0,0,0,0,0,0,0,9353,0
4,0,0,0,0,0,0,2756,0,0,0
5,0,3909,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,4887,0,0
8,16099,0,0,0,0,0,0,0,0,0
9,0,0,0,0,10374,0,0,0,0,0
A,0,0,0,0,0,15373,0,0,0,0


sex,Female,Male
subject_ID,,
1,0,7060
2,0,2379
3,0,9353
4,0,2756
5,0,3909
7,4887,0
8,0,16099
9,10374,0
A,0,15373


superinfection,Primary Only,Superinfection
subject_ID,,
1,7060,0
2,2379,0
3,9353,0
4,2756,0
5,3909,0
7,0,4887
8,16099,0
9,10374,0
A,0,15373


days_from_first_intubation,0,1,2
subject_ID,,,
1,0,0,7060
2,2379,0,0
3,0,9353,0
4,0,2756,0
5,0,3909,0
7,4887,0,0
8,16099,0,0
9,0,10374,0
A,15373,0,0


# Separate datasets

In [43]:
adata_cryo = adata[adata.obs['Sample status'].isin(['Cryo'])].copy()
adata_fresh = adata[adata.obs['Sample status'].isin(['Fresh'])].copy()

/home/icb/malte.luecken/anaconda3/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2492: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [44]:
adata_cryo.obs['dataset'] = [stu+'_cryo' for stu in adata_cryo.obs['study']]
adata_fresh.obs['dataset'] = [stu+'_fresh' for stu in adata_fresh.obs['study']]
del adata_cryo.obs['Sample status']
del adata_fresh.obs['Sample status']

In [45]:
adata_cryo
adata_fresh

AnnData object with n_obs × n_vars = 51689 × 21819
    obs: 'Day after intubation', 'No of SARS-CoV-2 genes', 'IFNA Response', 'IFNG Response', '% of mito genes', '% of ribo genes', 'subject_ID', 'sample', 'sample_type', 'original_celltype_ann', 'condition', 'n_counts', 'n_genes', 'study', 'age', 'sex', 'superinfection', 'days_from_first_intubation', 'dataset'
    var: 'gene_id'

AnnData object with n_obs × n_vars = 25457 × 21819
    obs: 'Day after intubation', 'No of SARS-CoV-2 genes', 'IFNA Response', 'IFNG Response', '% of mito genes', '% of ribo genes', 'subject_ID', 'sample', 'sample_type', 'original_celltype_ann', 'condition', 'n_counts', 'n_genes', 'study', 'age', 'sex', 'superinfection', 'days_from_first_intubation', 'dataset'
    var: 'gene_id'

# Write full datasets

In [47]:
adata_cryo.write('ready/full/grant_cryo.h5ad')
adata_fresh.write('ready/full/grant_fresh.h5ad')

# Gene selection

In [50]:
cd ../scripts/

/mnt/znas/icb_zstore01/groups/ml01/workspace/hlca_lisa.sikkema_malte.luecken/scripts


In [51]:
import preprocessing as pp


In [52]:
cd ../query_datasets/

/mnt/znas/icb_zstore01/groups/ml01/workspace/hlca_lisa.sikkema_malte.luecken/query_datasets


In [55]:
adata_cryo_sub = pp.subset_and_pad_adata(gene_set, adata_cryo)

not all genes were recovered, filling in 0 counts for 215 missing genes...


/home/icb/malte.luecken/anaconda3/lib/python3.7/site-packages/anndata/_core/merge.py:894: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  UserWarning,


In [56]:
adata_fresh_sub = pp.subset_and_pad_adata(gene_set, adata_fresh)

not all genes were recovered, filling in 0 counts for 215 missing genes...


In [58]:
adata_cryo_sub.write('../../../data/HLCA_extended/extension_datasets/ready/subsetted/grant_cryo_sub.h5ad')
adata_fresh_sub.write('../../../data/HLCA_extended/extension_datasets/ready/subsetted/grant_fresh_sub.h5ad')

... storing 'gene_id' as categorical
... storing 'gene_symbols' as categorical
... storing 'gene_id' as categorical
... storing 'gene_symbols' as categorical
